# Db2_on_Cloud_API_Load

こちらはQiita 「[Db2 on CloudのREST APIを使用したデータロード (python & curl)](https://qiita.com/nishikyon/items/cf45ffa83334444674dc)」で説明しているpython codeです。

# 1. 前準備

REST APIで使用するREST API host nameとCRNをあらかじめ取得しておきます。

詳細はQiita本編「[1. 前準備](https://qiita.com/nishikyon/items/cf45ffa83334444674dc#1-%E5%89%8D%E6%BA%96%E5%82%99)」を参照してください。

# 2.  アクセストークンの取得


### python前準備

冒頭で必要なライブラリと変数をセットしておきます。
<>で囲まれた部分は前準備で取得した値に置き換えてください。

In [ ]:
#必要なライブラリのimport
import requests
import json

#必要な変数のセット
REST_API_URL = "<1.1.3で取得したAPI hostname>"
CRN = "<1.2.3で取得したCRN>"
DB_USERID = "<dbのuserid>"
DB_PW = "<dbのpassword>"
LOAD_FILE = "<ロードするファイル名（PATH付き）>"
SCHEMA_NAME = "<ロード先テーブルのスキーマ名>"
TABLE_NAME = "<ロード先テーブル名>"

### 本編
実行が成功すると変数access_tokenにアクセストークンの値が入ります。


In [ ]:
#REST API の　　URL 作成
service_name = '/dbapi/v4/auth/tokens'
url = 'https://' + REST_API_URL + service_name

# request header作成
headers = {}
headers ['content-type'] = "application/json"
headers ['x-deployment-id']= CRN


# parameter dbアクセス用のuid, pwを指定
params = {}
params['userid'] = DB_USERID
params['password']= DB_PW

# Call the RESTful service
try:
    r = requests.post(url, headers=headers,  json=params)
except Exception as err:
    print("RESTful call failed. Detailed information follows.")
    print(err)
    sys.exit()

# Check for Invalid credentials
if (r.status_code == 401): # There was an error with the authentication
    print("RESTful called failed.")
    message = r.json()['errors']
    print(message)
    sys.exit()

# Check for anything other than 200/401
if (r.status_code != 200): # Some other failure
    print("RESTful called failed. Detailed information follows.")
    print(r.json())
    sys.exit()

# Retrieve the access token
try:
    access_token = r.json()['token']
    #print(r.json())
except:
    print("RESTful call did not return an access token.")
    print(r.json())
    sys.exit()

print (access_token)

# 3. ロードするファイル(CSVファイル)のアップロード

2の前準備、本編コードを実行して、必要なシェル変数、access_tokenにアクセストークンの値が入っていることが前提です。

実行が成功すると変数FILE_PATHにアップロードしたファイルのPATHの値が入ります。

In [ ]:
#REST API の　　URL 作成
service_name = '/dbapi/v4/home_content/'
url = 'https://' + REST_API_URL + service_name

# request header作成
headers = {}
headers ['authorization'] =  'Bearer ' + access_token #アクセストークンをHeaderにセット
headers ['x-deployment-id'] = CRN

# アップロードするファイルのセット
file = {'file': open( LOAD_FILE , 'rb')}


try:
    r = requests.post(url, headers=headers, files=file)
    print( r.status_code)
    #print(r.json())
except Exception as err:
    print("RESTful call failed. Detailed information follows.")
    print(err)


# Check for Invalid credentials
if (r.status_code == 401): # There was an error with the authentication
    print("RESTful called failed.")
    message = r.json()['errors']
    print(message)


# Check for anything other than 201/401
elif (r.status_code != 201): # Some other failure
    print("RESTful called failed. Detailed information follows.")
    print(r.text)
    #print(json.dumps(r.json(), indent=4))


# Print resppose
else:
    print(json.dumps(r.json(), indent=4))
    FILE_PATH = r.json()["resources"][0]["location"]
    print("FILE_PATH: " + FILE_PATH)
    

# 4. データロードジョブの作成

file_optionsとload_actionは必要に応じて変更してください。

注意点：

 - code_page: ファイルのコードページ, utf-8であれば1208 SJISは943 その他は[こちら](https://www.ibm.com/docs/ja/db2/11.1?topic=support-supported-territory-codes-code-pages)から
 - has_header_row: Header（1行目の項目名） なし　**no**のみセット可
 - load_action：　追加でINSERT INSERT / データの置換 REPLACE



file_optionsはWebコンソールから「データのロード」する際の、「定義」で指定するオプションと同じです。
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/294516/2b15713a-bc6b-c454-ea25-b4f136d03a0a.png)

2の前準備、本編、3のコードを実行して、必要なシェル変数、access_tokenにアクセストークンの値、　FILE_PATHの値が入っていること、ロードするファイルのアップロード実施済みが前提です。

実行が成功すると変数JOB_IDにアップロードしたデータロード　ジョブのIDの値が入ります。

In [ ]:
#REST API の　　URL 作成
service_name = '/dbapi/v4/load_jobs'
url = 'https://' + REST_API_URL + service_name

# request header作成
headers = {}
headers ['content-type'] = "application/json"
headers ['authorization'] =  'Bearer ' + access_token #アクセストークンをHeaderにセット
headers ['x-deployment-id'] = CRN

# ロード属性のセット
data = {
   "load_source": "SERVER",
   "schema": SCHEMA_NAME,
    "table":TABLE_NAME,
    "file_options":{
        "code_page":"1208", #ファイルのコードページ, utf-8であれば1208
        "has_header_row":"no", # Header（1行目の項目名） なし　no
        "column_delimiter":",", #カラムのデリミター
        "date_format":"YYYY-MM-DD", #Dateのフォーマット
        "time_format":"HH:MM:SS", #Timeのフォーマット
        "timestamp_format":"YYYY-MM-DD HH:MM:SS"}, #TimeStampのフォーマット
    "load_action": "INSERT", # 追加でINSERT INSERT / データの置換 REPLACE
    "server_source":{"file_path":FILE_PATH}  #アップロードしたファイルのPATH
}

try:
    r = requests.post(url, headers=headers, json=data)
    print( r.status_code)
except Exception as err:
    print("RESTful call failed. Detailed information follows.")
    print(err)


# Check for Invalid credentials
if (r.status_code == 401): # There was an error with the authentication
    print("RESTful called failed.")
    message = r.json()['errors']
    print(message)


# Check for anything other than 201/401
elif (r.status_code != 201): # Some other failure
    print("RESTful called failed. Detailed information follows.")
    print(r.text)
    #print(json.dumps(r.json(), indent=4))


# Print response
else:
    print(json.dumps(r.json(), indent=4))
    JOB_ID = r.json()["id"]
    print(f"JOB_ID: {JOB_ID}" )



# 5. データロードジョブの状況確認

2の前準備、本編、3,4のコードを実行して、必要なシェル変数、access_tokenにアクセストークンの値、　JOB_IDの値が入っていることが前提です。

In [ ]:
#REST API の　　URL 作成
service_name = '/dbapi/v4/load_jobs'
url = 'https://' + REST_API_URL + service_name + '/' +  str(JOB_ID)

# request header作成
headers = {}
headers ['content-type'] = "application/json"
headers ['authorization'] =  'Bearer ' + access_token #アクセストークンをHeaderにセット
headers ['x-deployment-id'] = CRN


try:
    r = requests.get(url, headers=headers)
    print( r.status_code)
except Exception as err:
    print("RESTful call failed. Detailed information follows.")
    print(err)


# Check for Invalid credentials
if (r.status_code == 401): # There was an error with the authentication
    print("RESTful called failed.")
    message = r.json()['errors']
    print(message)


# Check for anything other than 201/401
elif (r.status_code != 200): # Some other failure
    print("RESTful called failed. Detailed information follows.")
    print(r.text)
    #print(json.dumps(r.json(), indent=4))


# Print response
else:
    print(json.dumps(r.json(), indent=4))
    

